# `dask` avancé

Dans cette séance, vous découvrirez deux collections spécifiquement définies en `dask` : `dask.array` qui est un remplacement aux _numpy arrays_ et `dask.DataFrame` qui est la version `dask` des _dataframes_ pandas.

## Partie 1 : `dask.array`

La classe `dask.array` implémente un sous-ensemble des fonctionnalités des _numpy arrays_ à l'aide d'algorithmes par blocs, en découpant les tableaux trop grands en sous-tableaux.
Cela permet d'effectuer des calculs sur des tableaux plus grand que la mémoire disponible en utilisant tous les coeurs disponibles.
Ces calculs sont gérés, en interne, à l'aide de graphes de calcul Dask.

**Question.** En combien de blocs est divisé le `dask.array` créé par le code ci-dessous ?

In [ ]:
import numpy as np
import dask.array as da

a_da = da.ones(10, chunks=5)

Ainsi, le code ci-dessous :

In [ ]:
a_da.sum().compute()

est équivalent, si l'on se restreignait à utiliser des _numpy arrays_, à :

In [ ]:
a_np = np.ones(10)
a_np[:5].sum() + a_np[5:].sum()

(sauf que le code numpy implique de pouvoir stocker en mémoire le tableau en question).

**Question.** Quelle est la mémoire vive disponible sur votre machine ? Sachant qu'un entier est codé sur 32 bits, soit 4 octets, tentez d'initialiser un vecteur de 1 en numpy tel qu'il ne puisse pas tenir dans la mémoire de votre machine (faites par exemple en sorte que sa taille soit le double de votre RAM). Votre notebook plante-t-il ? Pourquoi ?

In [ ]:
%%time


**Question.** Faites la même chose en `dask` et comparez les temps d'exécution. Qu'est-ce qui justifie cette différence à votre avis ?

In [ ]:
%%time


**Question.** Calculez maintenant, en numpy d'une part et en dask d'autre part, la somme de ces vecteurs, et comparez les temps de calcul.

In [ ]:
%%time


In [ ]:
%%time


**Question.** Visualisez le graphe Dask de calcul de cette somme. Quelle semble être la stratégie utilisée pour calculer cette somme ?

La plupart des fonctions que vous connaissez en numpy sont également disponibles en Dask.

**Question.** Définissez une matrice `X` (au format _dask array_) de taille 30 000 x 30 000 d'éléments tirés selon une loi normale centrée réduite (vous utiliserez [`da.random.normal`](https://docs.dask.org/en/stable/generated/dask.array.random.normal.html)). Calculez la moyenne selon l'axe `axis=1` de la somme de `X` et de sa transposée.

### Influence de la taille des blocs

Il est possible de connaître la taille des blocs utilisés à l'aide de :

In [ ]:
X.chunksize

Ici, on a, sur chaque dimension, des blocs de taille 4096 (c'est-à-dire qu'ils sont constitués de sous-matrices de taille 4096x4096 sauf pour la dernière ligne/colonne qui peut recevoir des blocs plus petits).

On peut demander de changer cette taille avec :

In [ ]:
X = X.rechunk({0: 8192, 1: 8192})

**Question.** Évaluer l'influence de la taille des blocs sur l'efficacité du calcul ci-dessus.

In [ ]:
X = da.random.normal(0., 1., size=(30000, 30000))



In [ ]:
%%time



## Partie 2 : `dask.DataFrame`

Dans cette partie, vous allez manipuler des Data Frames `dask` pour mieux comprendre ce qui les différencie des Data Frames `pandas`.
Pour commencer, vous allez travailler sur des données issues du fichier `data/nycflights.tar.gz`.
Décompressez cette archive dans votre sous-dossier `data/` de manière à obtenir 10 fichiers CSV dans le dossier `data/nycflights`.

**Question.** Que fait le code suivant ?

In [ ]:
import pandas as pd
import os

filenames = [os.path.join("data", "nycflights", fname) 
             for fname in os.listdir(os.path.join("data", "nycflights")) 
             if fname.endswith(".csv")]
filenames

In [ ]:
%%time

sums = []
counts = []
for fn in filenames:
    # Read in file
    df = pd.read_csv(fn, 
                     parse_dates={"Date": [0, 1, 2]},
                     dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool})
    
    # Groupby origin airport
    by_origin = df.groupby('Origin')
    
    # Sum of all departure delays by origin
    total = by_origin.DepDelay.sum()
    
    # Number of flights by origin
    count = by_origin.DepDelay.count()
    
    # Save the intermediates
    sums.append(total)
    counts.append(count)

# Combine intermediates to get total mean-delay-per-origin
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights
mean

**Question.** Parallélisez ce code avec `dask.delayed`. Quel gain, en temps de calcul, obtenez-vous ?

In [ ]:
%%time

**Question.** Modifiez ce code pour utiliser un `dask.DataFrame`.

In [ ]:
import dask.dataframe as dd

In [ ]:
%%time

### Exercices

**Question.** Combien y a-t-il de lignes dans ce jeu de données ?

**Question.** Au total, combien y a-t-il de vols non annulés (attribut `Cancelled`) ?

**Question.** Au total, combien y a-t-il de vols non annulés (attribut `Cancelled`) au départ de chaque aéroport ?

**Question.** Quel est le retard moyen (attribut `DepDelay`) au départ de chaque aéroport ?

**Question.** Supposons que la colonne `Distance` soit erronnée et que vous deviez ajouter `1` à chaque valeur. Comment feriez-vous ?

**Question.** Quelle est la moyenne et l'écart-type du retard pour les vols non annulés en partance de l'aéroport `"JFK"` ?

Remarquez que pour ces deux calculs, on a besoin de se reposer sur le Data Frame des vols non annulés en partance de `"JFK"`. Si l'on a plus de calculs de ce type à effecter, il pourrait être intéressant de demander à Dask de charger effectivement ce sous-ensemble de données en mémoire.

**Question.** Étudier la documentation de la fonctionnalité [`persist()`](https://docs.dask.org/en/stable/best-practices.html#persist-when-you-can) et utilisez cette façon de faire.